In [11]:
import itertools
from collections import defaultdict
import dimod
from dimod import BQM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neal import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, LeapHybridSampler
from dwave.system import EmbeddingComposite
from IPython.display import display, HTML

In [5]:
def days_between(w1 , d1, w2, d2):
    if w1 > w2 or (w1 == w2 and d1 > d2):
        w1, w2, d1, d2 = w2, w1, d2, d1
    days = 7 * (w2 - w1)
    days = days + d2 - d1
    return days

In [47]:
def football_bqm(S, K, W, D, T, stadium):
    bqm = BQM("BINARY")
    P1 = 7
    P2 = 7
    P3 = 7
    P4 = 7
    P5 = 3
    P6 = 1
    P7 = 1
    P8 = 1
    
    """"
    #Add hard constraint (with slack variables)#1
    for k1, w, d in itertools.product(K, W, D):
        c1 = [(f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}", 1) for t, k2 in itertools.product(T, K) if k1 != k2]
        c1 = c1 + [(f"x_{stadium[k2]}_{k2}_{k1}_{w}_{d}_{t}", 1) for t, k2 in itertools.product(T, K) if k1 != k2]
        bqm.add_linear_inequality_constraint(
            c1, lagrange_multiplier=P1, ub=1, label=f"c1_{k1}_c_k2_k1_{w}_{d}_t")
    """
    #Add hard constraint#1
    for k1, w, d in itertools.product(K, W, D):
        games = [f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}" for t, k2 in itertools.product(T, K) if k1 != k2]
        games = games + [f"x_{stadium[k2]}_{k2}_{k1}_{w}_{d}_{t}" for t, k2 in itertools.product(T, K) if k1 != k2]
        for g1, g2 in itertools.product(range(len(games)), range(len(games))):
            if g1 < g2:
                bqm.add_quadratic(games[g1], games[g2], P1)
    
    #Add hard constraint #2
    for k1, k2 in itertools.product(K, K):
        if k1 != k2:
            c2 = [(f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}", 1) for w, d, t in itertools.product(W, D, T)]
            bqm.add_linear_equality_constraint(c2, lagrange_multiplier=P2, constant=-1)

    """
    #Add hard constraint (with slack variables)#3
    for s, w, d in itertools.product(S, W, D):
        c3 = [(f"x_{s}_{k1}_{k2}_{w}_{d}_{t}", 1) for t, k1, k2 in itertools.product(T, K, K) if k1 != k2 and s == stadium[k1]]
        bqm.add_linear_inequality_constraint(
            c3, lagrange_multiplier=P3, ub=1, label=f"c3_{s}_k1_k2_{w}_{d}_t")
        
    """
    #Add hard constraint #3
    for s, w, d in itertools.product(S, W, D):
        games = [f"x_{s}_{k1}_{k2}_{w}_{d}_{t}" for t, k1, k2 in itertools.product(T, K, K) if k1 != k2 and s == stadium[k1]]
        for g1, g2 in itertools.product(range(len(games)), range(len(games))):
            if g1 < g2:
                bqm.add_quadratic(games[g1], games[g2], P3)
    """    
     
    #Add hard constraint (with slack variables)#4
    for w, d, t in itertools.product(W, D, T):
        c4 = [(f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}", 1) for k1, k2 in itertools.product(K, K) if k1 != k2]
        bqm.add_linear_inequality_constraint(
            c4, lagrange_multiplier=P4, ub=1, label=f"c4_s_k1_k2_{w}_{d}_{t}")
        
    """
    #Add hard constraint #4
    for w, d, t in itertools.product(W, D, T):
        games = [f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}" for k1, k2 in itertools.product(K, K) if k1 != k2]
        for g1, g2 in itertools.product(range(len(games)), range(len(games))):
            if g1 < g2:
                bqm.add_quadratic(games[g1], games[g2], P4)
    
    #Add soft constraint #1
    for k1, k2, w, d, t in itertools.product(K, K, W, D, T):
        if k1 != k2 and d in ["P","O","Tr","C"]:
            bqm.add_linear(f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}", P5)
    
    #Add soft constraint#2
    for k in K:
        games = [ {"k1": k1, "k2": k2, "w": w, "d": d, "t": t} for k1, k2, w, d, t in itertools.product(K, K, W, range(len(D)), T) \
            if k1 != k2 and (k == k1 or k == k2)]
        for g1, g2 in itertools.product(range(len(games)), range(len(games))):
            if g1 < g2:
                gk1, gk2, gw1, gd1, gt1 = games[g1]["k1"], games[g1]["k2"], games[g1]["w"], games[g1]["d"], games[g1]["t"]
                gk3, gk4, gw2, gd2, gt2 = games[g2]["k1"], games[g2]["k2"], games[g2]["w"], games[g2]["d"], games[g2]["t"]
                if days_between(gw1, gd1, gw2, gd2) < 3:
                    bqm.add_quadratic(f"x_{stadium[gk1]}_{gk1}_{gk2}_{gw1}_{D[gd1]}_{gt1}",\
                                        f"x_{stadium[gk3]}_{gk3}_{gk4}_{gw2}_{D[gd2]}_{gt2}", P6)
                    
    #Add soft constraint #3
    for k1, k2, w, d, t in itertools.product(K, K, W, D, T):
        if k1 != k2 and d in ["P","O","Tr","C", "Pk"] and t < 18:
            bqm.add_linear(f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}", P7*(18 - t))
            
    #Add soft constraint #4
    for k1, k2, w, d, t in itertools.product(K, K, W, D, T):
        if k1 != k2 and d in ["S","Sv"] and t < 12:
            bqm.add_linear(f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}", P8*(12 - t))
    
    return bqm

In [7]:
def is_sample_feasible(sample, S, K, W, D, T, stadium):
    
    #Check hard constraint #1
    for k1, w, d in itertools.product(K, W, D):
        s1 = sum([sample[f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}"] for t, k2 in itertools.product(T, K) if k1 != k2])
        s1 = s1 + sum([sample[f"x_{stadium[k2]}_{k2}_{k1}_{w}_{d}_{t}"] for t, k2 in itertools.product(T, K) if k1 != k2])
        if s1 > 1:
            return False
    
    #Check hard constraint #2
    for k1, k2 in itertools.product(K, K):
        if k1 != k2:
            if sum([sample[f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}"] for w, d, t in itertools.product(W, D, T)]) != 1:
                return False
    
    #Check hard constraint #3
    for s, w, d in itertools.product(S, W, D):
        if sum([sample[f"x_{s}_{k1}_{k2}_{w}_{d}_{t}"] for t, k1, k2 in itertools.product(T, K, K)\
                if k1 != k2 and s == stadium[k1]]) > 1:
            return False
        
    #Check hard constraint #4
    for w, d, t in itertools.product(W, D, T):
        if sum([sample[f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}"] for k1, k2 in itertools.product(K, K) if k1 != k2]) > 1:
            return False
    
    return True

In [8]:
def get_scheedule(sample, S, K, W, D, T, stadium):
    
    scheedule = []
    for w in W:
        sc = [w]
        for d in D:
            if sum([sample[f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}"] for k1, k2, t in itertools.product(K, K, T) if k1 != k2]) == 0:
                sc.append("-")
            else:
                val = ""
                for t in T:
                    for k1, k2 in itertools.product(K, K):
                        if k1 != k2:
                            if sample[f"x_{stadium[k1]}_{k1}_{k2}_{w}_{d}_{t}"] == 1:
                                if val == "":
                                    val = f"{t}: {k1} - {k2}, {stadium[k1]}"
                                else:
                                    val = val + f"\n{t}: {k1} - {k2}, {stadium[k1]}"
                sc.append(val)
        scheedule.append(sc)
    display(HTML(pd.DataFrame(scheedule, columns = ["Week"] + D).to_html().replace("\\n","<br>")))

In [30]:
def prepare_medium_test():
    S = ["CampNou","SanSiro","Anfield", "Park de Prance"]
    K = ["Barcelona","Milan","Inter","Liverpool", "PSG"]
    W = [1,2, 3, 4, 5]
    D = ["P","O","Tr","C","Pk","S","Sv"]
    T = [10, 12, 14, 16, 18, 20]
    
    stadium = {"Barcelona": "CampNou", "Milan": "SanSiro", "Inter": "SanSiro", "Liverpool": "Anfield", "PSG": "Park de Prance"}
    
    return S, K, W, D, T, stadium


In [9]:
def prepare_small_test():
    S = ["CampNou","SanSiro"]
    K = ["Barcelona","Milan","Inter"]
    W = [1]
    D = ["P","O","Tr","C","Pk","S","Sv"]
    T = [16, 18, 20]
    
    stadium = {"Barcelona": "CampNou", "Milan": "SanSiro", "Inter": "SanSiro"}
    
    return S, K, W, D, T, stadium


In [45]:
def prepare_big_test():
    
    S = ["CampNou","SanSiro","Anfield", "Park de Prance", "Old Trafford",
        "Stamford Bridge", "Santiago Bernabeu"]
    K = ["Barcelona","Milan","Inter","Liverpool", "PSG", "Chelsea", "Real Madrid", "Manchester United"]
    W = [1,2, 3, 4, 5, 6, 7, 8, 9]
    D = ["P","O","Tr","C","Pk","S","Sv"]
    T = [8, 10, 12, 14, 16, 18, 20, 22]
    
    stadium = {"Barcelona": "CampNou", "Milan": "SanSiro", "Inter": "SanSiro", "Liverpool": "Anfield", "PSG": "Park de Prance",
               "Chelsea": "Stamford Bridge", "Real Madrid":"Santiago Bernabeu", "Manchester United": "Stamford Bridge"}
    
    return S, K, W, D, T, stadium

In [48]:
#S, K, W, D, T, stadium = prepare_big_test()
S, K, W, D, T, stadium  = prepare_medium_test()
#S, K, W, D, T, stadium = prepare_small_test()
    
bqm = football_bqm(S, K, W, D, T, stadium)
len(bqm.variables)

4200

## Simulated Annealing

Can handle problem instances of any size

In [49]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads = 300)

print(sampleset)

    ... x_SanSiro_Milan_PSG_5_Tr_20 energy num_oc.
12  ...                           0   19.0       1
18  ...                           0   19.0       1
61  ...                           0   19.0       1
165 ...                           0   19.0       1
214 ...                           0   19.0       1
280 ...                           0   19.0       1
282 ...                           0   19.0       1
25  ...                           0   20.0       1
26  ...                           0   20.0       1
38  ...                           0   20.0       1
100 ...                           0   20.0       1
102 ...                           0   20.0       1
181 ...                           0   20.0       1
223 ...                           0   20.0       1
261 ...                           0   20.0       1
266 ...                           0   20.0       1
269 ...                           0   20.0       1
5   ...                           0   21.0       1
6   ...                        

In [50]:
best_sample = sampleset.first.sample
is_feasible = is_sample_feasible(best_sample, S, K, W, D, T, stadium)
print(is_feasible)
get_scheedule(best_sample, S, K, W, D, T, stadium)

True


,Week,P,O,Tr,C,Pk,S,Sv
0,1,"18: Milan - Inter, SanSiro",-,-,"18: Barcelona - PSG, CampNou","20: Inter - Liverpool, SanSiro","12: Inter - Milan, SanSiro","14: Liverpool - PSG, Anfield"
1,2,-,-,-,-,"20: Inter - Barcelona, SanSiro","14: Milan - PSG, SanSiro","16: Milan - Barcelona, SanSiro18: Liverpool - Inter, Anfield"
2,3,-,-,-,-,"18: Barcelona - Milan, CampNou","12: PSG - Inter, Park de Prance","14: Liverpool - Barcelona, Anfield"
3,4,-,-,-,-,"18: Milan - Liverpool, SanSiro","16: Barcelona - Liverpool, CampNou20: Inter - PSG, SanSiro",-
4,5,-,"20: PSG - Barcelona, Park de Prance",-,-,"18: PSG - Milan, Park de Prance","20: PSG - Liverpool, Park de Prance","12: Barcelona - Inter, CampNou20: Liverpool - Milan, Anfield"


## Quantum Annealing
First of all we create quantum and hybid solvers, by providing Leap subscription token

In [17]:
qpu_advantage = EmbeddingComposite(DWaveSampler(solver={'topology__type': 'pegasus'}, token="DEV-fc61310ff217929feac5a5ec87b1b5e23302c79a"))
hybrid_bqm_sampler = LeapHybridSampler(token="DEV-fc61310ff217929feac5a5ec87b1b5e23302c79a")

### Hybrid solver
Can be used for problem instances of any size

In [51]:
hybrid_sampleset = hybrid_bqm_sampler.sample(bqm)
print(hybrid_sampleset)

  ... x_SanSiro_Milan_PSG_5_Tr_20 energy num_oc.
0 ...                           0   15.0       1
['BINARY', 1 rows, 1 samples, 4200 variables]


In [52]:
best_hybrid_sample = hybrid_sampleset.first.sample
is_feasible = is_sample_feasible(best_hybrid_sample, S, K, W, D, T, stadium)
print(is_feasible)
get_scheedule(best_hybrid_sample, S, K, W, D, T, stadium)

True


,Week,P,O,Tr,C,Pk,S,Sv
0,1,-,-,-,-,"18: Liverpool - Inter, Anfield","16: Milan - Barcelona, SanSiro","14: PSG - Inter, Park de Prance16: Milan - Liverpool, SanSiro"
1,2,-,-,-,-,"18: PSG - Milan, Park de Prance","14: Barcelona - Inter, CampNou","12: Liverpool - Milan, Anfield20: PSG - Barcelona, Park de Prance"
2,3,-,-,-,-,"18: Barcelona - PSG, CampNou20: Inter - Liverpool, SanSiro",-,"16: Milan - Inter, SanSiro20: Liverpool - Barcelona, Anfield"
3,4,-,-,-,-,"18: Milan - PSG, SanSiro","18: PSG - Liverpool, Park de Prance20: Inter - Milan, SanSiro","16: Inter - Barcelona, SanSiro"
4,5,-,-,-,-,"18: Inter - PSG, SanSiro20: Barcelona - Liverpool, CampNou","20: Barcelona - Milan, CampNou","14: Liverpool - PSG, Anfield"


### Quantum solver
Can be used only for small problem instances with size of ~100 variables 

In [20]:
quantum_sampleset = qpu_advantage.sample(bqm, num_reads = 100)
print(quantum_sampleset)

   ... x_SanSiro_Milan_Inter_1_Tr_20 energy num_oc. chain_b.
20 ...                             0   31.0       1 0.190476
3  ...                             0   36.0       1 0.166667
5  ...                             0   41.0       1  0.15873
96 ...                             0   41.0       1 0.230159
8  ...                             0   42.0       1 0.206349
86 ...                             0   42.0       1 0.222222
56 ...                             1   49.0       1 0.246032
13 ...                             0   50.0       1 0.174603
24 ...                             0   60.0       1  0.18254
14 ...                             1   64.0       1 0.198413
28 ...                             0   65.0       1 0.190476
34 ...                             0   68.0       1 0.166667
29 ...                             0   69.0       1 0.190476
37 ...                             0   74.0       1 0.214286
11 ...                             0   75.0       1 0.190476
0  ...                  

In [21]:
best_quantum_sample = quantum_sampleset.first.sample
is_feasible = is_sample_feasible(best_quantum_sample, S, K, W, D, T, stadium)
print(is_feasible)
get_scheedule(best_quantum_sample, S, K, W, D, T, stadium)

False


,Week,P,O,Tr,C,Pk,S,Sv
0,1,-,"20: Inter - Milan, SanSiro","16: Barcelona - Inter, CampNou","20: Milan - Barcelona, SanSiro","16: Milan - Inter, SanSiro20: Barcelona - Milan, CampNou",-,"18: Inter - Barcelona, SanSiro"
